# TUTORIAL

In this notebook, we are going to have a good tour of main features, usages and how-to of the package.

## 1. Install

As requested we need to install the package from pypi. Depends of your internet connection, it could take 1 or 2 minutes.

In [1]:
!pip install legal_doc_processing==2.2.3

## 2.Check package integriity and download third parties objects

As we will use major NLP Librairies,we need to download data collections and mandory web assets such as NLTK stop words, Scapy or Transformers models or tensorflow/pyTorch assets. It can take 1/2 minutes (depends of your web connection)

In [2]:
import legal_doc_processing.boot
boot.boot()

NameError: name 'boot' is not defined

## 3. Understand bascic package structure

There is 3 main modules in legal-doc-processing : 
- legal_doc for LegalDoc objects ie order, complaint, etc etc all kinds of official documents
- press_release for PressRelease objects for legal press release related to each case
- decision for both LegalDoc and PressRelease documents. The Decision object is able to read both legal document and press release, make prediction from both documents and merge/clean predictions of both documents.


so you can :

In [ ]:
from legal_doc_processing import legal_doc # import legal document module
from legal_doc_processing import press_release # import press release module
from legal_doc_processing import decision # import decision module

each package has its own Object

In [ ]:
print(f"press_release.PressRelease : {press_release.PressRelease} \n")
print(f"press_release.__doc__ : {press_release.__doc__} \n")
print(f"press_release.__dict__ : {press_release.__dict__}")

as consequence it is recommanded to use the import method : 

In [4]:
from legal_doc_processing import press_release as pr

print(pr.PressRelease)

<class 'legal_doc_processing.press_release.press_release.PressRelease'>


LegalDoc, PressRelease and Decision are inheritance of The Base class. You can consider The Base class is an
as an abstraction of 3 others objecst.

Of course you can call this class, even if you will not mainuplate it 

In [ ]:
from legal_doc_processing.base.base import Base

print(Base)

## 4 Object instanciation

youn can init an object in 3 ways.: 
- given a string
- given a text file path
- give an url

this three methods are onboarded feature form the package: 



In [7]:
# with a string

pr.PressRelease
pr.from_text

print(pr.PressRelease)
print(pr.from_text)

AttributeError: type object '_PressRelease' has no attribute 'from_text'

In [8]:
# with a file

pr.from_file
print(pr.from_file)

SyntaxError: invalid syntax (<ipython-input-8-b5af798b07bb>, line 3)

In [10]:
# with a url

pr.from_url
print(pr.from_url)

AttributeError: type object '_PressRelease' has no attribute 'from_url'

In [ ]:

<br>

### Instanciation
youn can init an object in 3 ways.
* with text directly
```
from legal_doc_processing import ld
doc = ld.LegalDoc("this is a document")
# or better
doc = ld.from_text("this is a document")
```
* with path to a file
```
doc = ld.from_file("this/is/my/file.txt")
```

for press release, same pattern: 
```
from legal_doc_processing import pr
press = ld.PressRelease("this is a press release")
press = ld.from_text("this is a press release")
```
* with path to a file
```
press = ld.from_file("this/is/my/file.txt")
```

once instanciated, you can print : 
```
print(doc)
```
<br>

all interessing features are in feature_dict attribute
```
print(doc.feature_dict)
```
<br>

### Predictions

you can now make predictions : 
```
defendant = doc.predict("defendant")
print(defendant)
case = doc.predict("case")
print(case)
```
<br>

predict method return a something but alson work on the object itself : 
```
print(doc)
print(doc.feature_dict)
```
<br>

of course most easy is to predict all : 
```
features = doc.predict("all")
print(features)
print(doc)
```
<br>

you can acces to final predictions or detailed prediction: 
```
print(doc.feature_dict) # just the results
print(doc._feature_dict) # results + score for each result
```

<br>
<br>